In [4]:
import rasterio as rio
import xarray as xr
import numpy as np
from rasterio.windows import Window
from pyproj import Transformer,CRS,Proj
from rasterio.warp import transform
import rioxarray
import math
import csv
cesm_temp_data = xr.open_dataset('/Users/gclyne/tas_Amon_CESM2_land-hist_r1i1p1f1_gn_185001-201512.nc')
output = open('boreal_observed_output.csv', 'w')
csv_writer = csv.writer(output)
cesm_temp_data = cesm_temp_data.isel(lat=np.logical_and(cesm_temp_data.lat<90,cesm_temp_data.lat > 30))
cesm_temp_data = cesm_temp_data.isel(lon=np.logical_and(cesm_temp_data.lon-360>-170,cesm_temp_data.lon-360 <-40))

transformer =Transformer.from_crs(4326,3978)
obs_tree = rioxarray.open_rasterio('/Users/gclyne/Downloads/CA_forest_VLCE2_1984/CA_forest_VLCE2_1984.tif')
other_transformer = Transformer.from_crs(3978,4326)
for lat_index in range(len(cesm_temp_data['lat'].values) - 1):
    for lon_index in range(len(cesm_temp_data['lon'].values) - 1):
        lat = cesm_temp_data['lat'].data[lat_index]
        lon = cesm_temp_data['lon'].data[lon_index]
        next_lat = cesm_temp_data['lat'].data[lat_index+1]
        next_lon = cesm_temp_data['lon'].data[lon_index+1]
        lon = lon - 360 if lon > 180 else lon # this is not the right formula, does not cover edge cases eg. 178 goes to -180
        next_lon = next_lon - 360 if next_lon > 360 else next_lon
        lat,lon = transformer.transform(lat,lon)
        # print(obs_tree['x'].data[0],obs_tree['y'].data)
        # print(other_transformer.transform(obs_tree['x'].data[-1],obs_tree['y'].data[-1]))
        next_lat,next_lon = transformer.transform(next_lat,next_lon)
        # print(lat,lon,next_lat,next_lon)
        # print(obs_tree)
        # print(obs_tree.isel(band=0, x=obs_tree.x >= lon, y=obs_tree.y >= lat))
        sl = obs_tree.isel(
        x=np.logical_and(obs_tree.x >= lat, obs_tree.x <= next_lat),
        y=np.logical_and(obs_tree.y >= lon, obs_tree.y <= next_lon),
        band=0
        )
        # print(len(sl))
        # if(len(sl) == 0):
        #     continue
        # print(sl)
        # print(sl.data)
        try: 
            tree_coverage = sl.where(np.isin(sl.data,[230,220,210,81]))
            tree_coverage = tree_coverage.groupby('x')
            tree_coverage = tree_coverage.count('y')
            tree_coverage = tree_coverage.sum()
            lat,lon = other_transformer.transform(lat,lon)
            csv_writer.writerow([lat,lon,tree_coverage.values / sl.size * 100])
            print(lat,lon,tree_coverage.values / sl.size * 100)
        except ValueError:
            pass


/Users/gclyne/.pyenv/versions/3.9.1/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


33.45549774169921 -62.50000000000001 0.0
34.39790725708009 -65.0 0.0
34.39790725708009 -63.75000000000001 0.0
34.39790725708009 -62.50000000000001 0.0
35.340312957763665 -67.50000000000001 0.0
35.34031295776368 -66.25 0.0
35.34031295776368 -65.0 0.0
35.34031295776368 -63.75000000000001 0.0
35.34031295776368 -62.50000000000001 0.0
35.34031295776368 -61.25000000000001 0.0
36.282722473144545 -125.00000000000001 0.0
36.282722473144545 -123.75000000000001 0.0
36.282722473144545 -122.50000000000001 0.0
36.282722473144545 -70.0 0.0
36.282722473144545 -68.75 0.0
36.282722473144545 -67.50000000000001 0.0
36.282722473144545 -66.25 0.0
36.282722473144545 -65.0 0.0
36.282722473144545 -63.75000000000001 0.0
36.282722473144545 -62.50000000000001 0.0
36.282722473144545 -61.25000000000001 0.0
37.22513198852539 -125.00000000000001 0.0
37.22513198852539 -123.75000000000001 0.0
37.22513198852539 -122.50000000000001 0.0
37.22513198852539 -121.24999999999999 0.0
37.22513198852541 -119.99999999999999 0.0
37